In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np
from xtquant import xtdatacenter as xtdc

from vnpy.trader.utility import TEMP_DIR

In [ ]:
# 设置迅投研令牌
xtdc.set_token("123456")    # 换成自己的Token

# 设置临时文件目录
xtdc.set_data_home_dir(str(TEMP_DIR) + "\\xt")

# 初始化连接
xtdc.init()

In [ ]:
from xtquant.xtdata import (
    download_history_data,
    get_local_data
)

In [ ]:
def download_qlib_data(xt_symbol: str, start: datetime, end: datetime) -> pd.DataFrame:
    start = start.strftime("%Y%m%d")
    end = end.strftime("%Y%m%d")

    # 执行下载数据
    download_history_data(xt_symbol, "1d", start , end)

    # 获取原始价格
    original_data = get_local_data([], [xt_symbol], "1d", start , end, dividend_type="none")
    original_df = original_data[xt_symbol]

    # 获取前复权价格
    adjusted_data = get_local_data([], [xt_symbol], "1d", start , end, dividend_type="front")
    adjusted_df = adjusted_data[xt_symbol]

    if adjusted_df.empty:
        print("fuck", xt_symbol, start, end)
        print(original_data)
        print(adjusted_data)

    # 转换前复权价格
    first_close = adjusted_df.iloc[0]["close"]

    result_df = adjusted_df[["time", "open", "high", "low", "close", "volume"]].copy()
    result_df.index = pd.DatetimeIndex(result_df.index)

    result_df["open"] = result_df["open"] / first_close
    result_df["high"] = result_df["high"] / first_close
    result_df["low"] = result_df["low"] / first_close
    result_df["close"] = result_df["close"] / first_close
    result_df["volume"] = result_df["volume"] * first_close

    result_df["factor"] = result_df["close"] / original_df["close"]

    return result_df

In [ ]:
# 读取日历信息
dates = []
with open("day.txt", mode="r") as f:
    for line in f.readlines():
        dates.append(line.replace("\n", ""))


def dump_bin_file(symbol: str, df: pd.DataFrame):
    instrument_folder = folder = Path.cwd().joinpath("instruments")
    if not instrument_folder.exists():
        instrument_folder.mkdir()

    folder = instrument_folder.joinpath(symbol)
    folder.mkdir()

    first_date: str = df.index[0].strftime("%Y-%m-%d")
    ix: int = dates.index(first_date)

    # 保存到bin文件
    for field in ["open", "high", "low", "close", "volume", "factor"]:
        file_path = folder.joinpath(f"{field}.day.bin")

        with open(str(file_path), mode="wb") as fp:
            np.hstack([ix, df[field]]).astype("<f").tofile(fp)

In [ ]:
def load_qlib_symbols():
    symbols = []

    with open("csi300.txt", mode="r") as f:
        for line in f.readlines():
            qlib_symbol = line.split("\t")[0]
            symbols.append(qlib_symbol)

    return symbols

In [ ]:
# 下载数据
start = datetime(2024, 1, 1)
end = datetime(2024, 2, 21)

qlib_symbols = load_qlib_symbols()

for qlib_symbol in qlib_symbols:
    xt_symbol = qlib_symbol[2:] + "." + qlib_symbol[:2]

    df = download_qlib_data(xt_symbol, start, end)
    dump_bin_file(qlib_symbol, df)